In [ ]:
!pip install nltk
!pip install tensorflow==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 40.5 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  At

In [ ]:
import json
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [ ]:
with open('intents.json', 'r') as file:
    data = json.load(file)

In [ ]:
# Download the 'punkt_tab' resource
nltk.download('punkt_tab') 
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(f"{len(documents)} documents")
print(f"{len(classes)} classes: {classes}")
print(f"{len(words)} unique lemmatized words: {words}")


639 documents
91 classes: ['about', 'afternoon', 'ask', 'bad_friendship', 'casual', 'creation', 'death', 'default', 'depression', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'financial_anxiety', 'future_anxiety', 'general_anxiety', 'good_friendship', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'health_anxiety', 'help', 'jokes', 'learn-mental-health', 'learn-more', 'location', 'lonely', 'lost_friendship', 'meditation', 'mental-health-fact', 'morning', 'name', 'neutral-response', 'night', 'no-approach', 'no-response', 'no_friendship', 'not-talking', 'performance_anxiety', 'problem', 'relationship_anxiety', 'repeat', 'sad', 'scared', 'skill', 'sleep', 'social_anxiety', 'somet

In [ ]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Eğitim verisi hazır!")


Eğitim verisi hazır!


In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5")
print("Model başarıyla kaydedildi!")


Epoch 1/200
128/128 [==============================] - 1s 3ms/step - loss: 4.4822 - accuracy: 0.0329
Epoch 2/200
128/128 [==============================] - 0s 3ms/step - loss: 4.2911 - accuracy: 0.0579
Epoch 3/200
128/128 [==============================] - 0s 3ms/step - loss: 4.1275 - accuracy: 0.0704
Epoch 4/200
128/128 [==============================] - 0s 3ms/step - loss: 3.9060 - accuracy: 0.0892
Epoch 5/200
128/128 [==============================] - 0s 3ms/step - loss: 3.6720 - accuracy: 0.1158
Epoch 6/200
128/128 [==============================] - 0s 3ms/step - loss: 3.5614 - accuracy: 0.1440
Epoch 7/200
128/128 [==============================] - 1s 4ms/step - loss: 3.2894 - accuracy: 0.1768
Epoch 8/200
128/128 [==============================] - 0s 2ms/step - loss: 3.1550 - accuracy: 0.2019
Epoch 9/200
128/128 [==============================] - 0s 2ms/step - loss: 2.9470 - accuracy: 0.2426
Epoch 10/200
128/128 [==============================] - 0s 2ms/step - loss: 2.8349 - accura

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("chatbot_model.h5")

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words):
    bag = [0] * len(words)
    sentence_words = clean_up_sentence(sentence)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)

    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intent_list, intents_json):
    if not intent_list:  # Eğer intent_list boşsa
        return "I'm sorry, I didn't understand that. Could you rephrase?"

    tag = intent_list[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            return random.choice(i['responses'])
    return "I'm sorry, I didn't understand that."

# Kullanıcı girdilerini test etmek için
print("Chatbot hazır! Sorularınızı sorabilirsiniz.")
while True:
    message = input("You: ")
    intents = predict_class(message, model)
    response = get_response(intents, data)
    print("Bot:", response)


Chatbot hazır! Sorularınızı sorabilirsiniz.
You: hello
1/1 [==============================] - 0s 90ms/step
Bot: Hi there. What brings you here today?
You: I wish someone was here to talk to me right now
1/1 [==============================] - 0s 24ms/step
Bot: Feeling empty is hard. Let’s talk about what brings you even a little bit of joy.
You: I wish someone was here to talk to me right now
1/1 [==============================] - 0s 19ms/step
Bot: Reaching out to someone can be difficult, but it might be worth a try. Is there anyone you trust?
You: Nothing makes sense anymore
1/1 [==============================] - 0s 20ms/step
Bot: It's only natural to feel this way. Tell me more. What else is on your mind?
You: Nothing makes sense anymore
1/1 [==============================] - 0s 19ms/step
Bot: I first want to let you know that you are not alone in your feelings, and there is always someone there to help. You can change your feelings and your way of thinking by being open to trying.
Y

KeyboardInterrupt: Interrupted by user